In this notebook we can run motion correction to get an estimate of motion.

In [ ]:


import spikeinterface.full as si
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

job_kwargs = dict(n_jobs=-1, chunk_duration='1s', progress_bar=True) # how to chunk and process data

# specify the folder
base_folder = Path(r'C:\SGL_DATA\Group4_bank0_kc\KS001_V1_011526_KC_g0')

spikeglx_folder = base_folder
# load spikeglx
stream_names, stream_ids = si.get_neo_streams('spikeglx', spikeglx_folder)

raw_rec = si.read_spikeglx(spikeglx_folder, stream_name=stream, load_sync_channel=False)

rec_corrected = correct_motion(recording=rec, preset="rigid_fast")



Check the instructions [**here**](https://spikeinterface.readthedocs.io/en/0.93.0/overview.html).

Be sure to checkout the other [tutorials](https://spikeinterface.readthedocs.io/en/0.93.0/getting_started/plot_getting_started.html#) and the [how to guide](https://spikeinterface.readthedocs.io/en/latest/how_to/analyze_neuropixels.html). 


In [ ]:
import spikeinterface.full as si
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# parameters
FREQ_LOW = 300 # Frequency of the bandpass filter
ks4_params = si.get_default_sorter_params('kilosort4')
ks4_params['do_CAR'] = False # skip CAR in kilosort
job_kwargs = dict(n_jobs=-1, chunk_duration='1s', progress_bar=True) # how to chunk and process data

# specify the folder
base_folder = Path(r'C:\SGL_DATA\Group4_bank0_kc\KS001_V1_011526_KC_g0')
spikeglx_folder = base_folder
# load spikeglx
stream_names, stream_ids = si.get_neo_streams('spikeglx', spikeglx_folder)

print(f'There are streams {stream_names} in folder {spikeglx_folder}')


In [ ]:
print(f'All Kilosort4 parameters are: ')
for k in ks4_params.keys():
    print(f'   - {k}: {ks4_params[k]}')

In [ ]:
# lets sort all 'ap streams'
for stream in stream_names:
    if not stream.endswith('.ap'):
        continue    
    raw_rec = si.read_spikeglx(spikeglx_folder, stream_name=stream, load_sync_channel=False)
    # frequency filter
    rec1 = si.highpass_filter(raw_rec, freq_min=FREQ_LOW)
    bad_channel_ids, channel_labels = si.detect_bad_channels(rec1)
    rec2 = rec1.remove_channels(bad_channel_ids)
    print('bad_channel_ids', bad_channel_ids)
    # phase shift
    rec3 = si.phase_shift(rec2)
    # CAR
    rec = si.common_reference(rec3, operator="median", reference="global")
    # save to disk
    rec = rec.save(folder=base_folder / 'preprocess', format='binary', **job_kwargs)
    sort_folder = base_folder / 'kilosort4_output'
    # run ks4
    sorting = si.run_sorter('kilosort4', rec, folder=sort_folder,
                        docker_image=False, verbose=True, **ks4_params)
    sorting = si.read_sorter_folder(sort_folder)

    analyzer = si.create_sorting_analyzer(sorting, rec, sparse=True, format="memory")
    # compute waveforms 
    analyzer.compute("random_spikes", method="uniform", max_spikes_per_unit=500)
    analyzer.compute("waveforms",  ms_before=1.5,ms_after=1.5, **job_kwargs)
    analyzer.compute("templates", operators=["average", "median", "std"])
    analyzer.compute("noise_levels")
    analyzer.compute("correlograms")
    analyzer.compute("unit_locations")
    analyzer.compute("spike_amplitudes", **job_kwargs)
    # save
    analyzer_saved = analyzer.save_as(folder=base_folder / "analyzer", format="binary_folder")

    metric_names=['firing_rate', 'presence_ratio', 'snr', 'isi_violation', 'amplitude_cutoff']
    metrics = si.compute_quality_metrics(analyzer, metric_names=metric_names)
    from spikeinterface.exporters import export_to_phy
    export_to_phy(sorting_analyzer=analyzer, output_folder=base_folder/'phy_output')

bad_channel_ids ['imec0.ap#AP72' 'imec0.ap#AP145' 'imec0.ap#AP238' 'imec0.ap#AP239'
 'imec0.ap#AP334' 'imec0.ap#AP335']
write_binary_recording 
engine=process - n_jobs=24 - samples_per_chunk=30,000 - chunk_memory=21.63 MiB - total_memory=519.10 MiB - chunk_duration=1.00s


write_binary_recording (workers: 24 processes):   0%|          | 0/601 [00:00<?, ?it/s]

kilosort.run_kilosort:  
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: N samples: 18019149
kilosort.run_kilosort: N seconds: 600.6383
kilosort.run_kilosort: N batches: 301


Skipping common average reference.


kilosort.run_kilosort: Preprocessing filters computed in 1.85s; total 1.85s
kilosort.run_kilosort:  
kilosort.run_kilosort: Resource usage after preprocessing
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort: CPU usage:     7.90 %
kilosort.run_kilosort: Mem used:     36.90 %     |      11.57 GB
kilosort.run_kilosort: Mem avail:    19.80 / 31.36 GB
kilosort.run_kilosort: ------------------------------------------------------
kilosort.run_kilosort: GPU usage:    `conda install pynvml` for GPU usage
kilosort.run_kilosort: GPU memory:   36.08 %     |      2.87   /     7.96 GB
kilosort.run_kilosort: Allocated:     0.11 %     |      0.01   /     7.96 GB
kilosort.run_kilosort: Max alloc:    16.16 %     |      1.29   /     7.96 GB
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing drift correction.
kilosort.run_kilosort: -----------------------------------

...

c:\Users\np_user\miniforge3\envs\npix-analysis\Lib\site-packages\spikeinterface\qualitymetrics\misc_metrics.py:1059: UserWarning: Some units have too few spikes : amplitude_cutoff is set to NaN
  warnings.warn(f"Some units have too few spikes : amplitude_cutoff is set to NaN")


write_binary_recording (no parallelization):   0%|          | 0/601 [00:00<?, ?it/s]

Fitting PCA:   0%|          | 0/329 [00:00<?, ?it/s]

Projecting waveforms:   0%|          | 0/329 [00:00<?, ?it/s]

extract PCs (no parallelization):   0%|          | 0/601 [00:00<?, ?it/s]